# Replace Non-Task commands with Dynamic Tables

This notebook identifies `CTAS` and `INSERT OVERWRITE` commands executed multiple times over a given time frame (not via a task), that can be converted to Dynamic tables.  Dynamic tables simplify data engineering in Snowflake by providing a reliable, cost-effective, and automated way to transform data. Not every command can or should be replaced.  

This notebook will:
- check the `QUERY_HISTORY` account usage view for the commands that have successfully completed, more than once, over the last 24 hours.
- identify whether the current target table is in a share.
    - **NOTE:** Data Providers should take additional steps to ensure any affected shared tables don't impact Consumers before switching to Dynamic tables.
- generate the DDL to create the Dynamic table that will replace the commands
- execute the Dynamic table DDL (optional)
- remove the existing target table from the share, if applicable (optional)
- drop the existing target table (optional)

## Prerequisites:

- The user executing this notebook, must have access to the `SNOWFLAKE` database.
- The user must have the `CREATE DYNAMIC TABLE` privilge on the schema where the new Dynamic Table will be created.

## STEP 1: Initiaize Session

In [ ]:
import json
import numpy as np
import pandas as pd
import re
import streamlit as st
from st_aggrid import AgGrid, GridUpdateMode, JsCode
from st_aggrid.grid_options_builder import GridOptionsBuilder
import sqlparse

session = get_active_session()

#tag session
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"dt_conversion_non_task","version":{{"major":1, "minor":0}},"attributes":"session_tag"}}'""").collect()

st.success(f"Session initialized 🎉")

## STEP 2: Function definition

In [ ]:
def paginate_data(df):
	st.divider()
			
	pagination = st.empty()
	batch_size = 20  # Set the number of items per page
 
	bottom_menu = st.columns((4, 1, 1))
	with bottom_menu[2]:
		total_pages = (
			int(len(df) / batch_size) if int(len(df) / batch_size) > 0 else 1
		)
		current_page = st.number_input(
			"Page", min_value=1, max_value=total_pages, step=1
		)
	with bottom_menu[0]:
		st.markdown(f"Page **{current_page}** of **{total_pages}** ")

	pages = split_frame(df, batch_size)
	pagination.dataframe(data=pages[current_page - 1], use_container_width=True)

	st.divider()

@st.cache_data(show_spinner=False)
def split_frame(input_df, rows):
	df = [input_df.loc[i : i + rows - 1, :] for i in range(0, len(input_df), rows)]
	return df

st.success(f"Functions created 🎉")

## STEP 3: Get all shared tables/views

This step compiles a list of all tables/views shared by your role.

In [ ]:
list_shares_objs = []

#show all shares
session.sql(f"""SHOW SHARES;""").collect()

#get outbound shares only
df_outbound_shares = pd.DataFrame(session.sql(f"""SELECT "name" FROM TABLE(RESULT_SCAN(LAST_QUERY_ID())) WHERE LOWER("kind") = 'outbound';""").collect())

for index, row in df_outbound_shares.iterrows():
    share = row["name"]

    #describe shares
    session.sql(f"""DESCRIBE SHARE {share};""").collect()
    
    #get shared objects
    df_shared_objs = pd.DataFrame(session.sql(f"""SELECT "name", "kind" FROM TABLE(RESULT_SCAN(LAST_QUERY_ID())) 
                                                    WHERE 
                                                        LOWER("kind") IN ('table', 'view', 'materialized view');""").collect())

    #add each object to the list_obj list
    for index, row in df_shared_objs.iterrows():
        name = row["name"]
        kind = row["kind"]

        if not row.empty:
            list_shares_objs.append([share, kind, name])

#create list of shares, object types, and objs shared
list_shares = [item[0] for item in list_shares_objs]
list_obj_types = [item[1] for item in list_shares_objs]
list_objs = [item[2] for item in list_shares_objs]
   
df_shared_objs = pd.DataFrame({'share': list_shares, 'object_type': list_obj_types, 'object': list_objs} )

#show shared objects
paginate_data(df_shared_objs)



## STEP 4: Find the completed CTAS/INSERT OVERWRITE commands.

This step compiles a list of latest `CTAS` and `INSERT OVERWRITE` commands completed within the last 24 hours.

In [ ]:
query_history_range_list = ['Choose a Date Range', 'Last day', 'Last 7 days', 'Last 28 days', 'Last 3 months', 'Last 6 months', 'Last 12 months']
st.write("")
st.selectbox("Select Query History Date Range:", query_history_range_list, key="sb_query_history_range")

if st.session_state.sb_query_history_range in ['Last 3 months', 'Last 6 months', 'Last 12 months']:
    st.warning('Consider using either a larger warehouse or container for this Date Range', icon="⚠️")

date_time_part = ""
increment = ""
df_query_history_range = None

#match st.session_state.sb_query_history_range:
if st.session_state.sb_query_history_range == "Last day":
    date_time_part = "hours"
    increment = "24"
elif st.session_state.sb_query_history_range == "Last 7 days":
    date_time_part = "days"
    increment = "7"
elif st.session_state.sb_query_history_range == "Last 28 days":
    date_time_part = "days"
    increment = "28"
elif st.session_state.sb_query_history_range == "Last 3 months":
    date_time_part = "months"
    increment = "3"
elif st.session_state.sb_query_history_range == "Last 6 months":
    date_time_part = "months"
    increment = "6"
elif st.session_state.sb_query_history_range == "Last 12 months":
    date_time_part = "months"
    increment = "12"

if st.session_state.sb_query_history_range == "Choose a Date Range":
    st.write("#")
    st.write("#")
    st.write("#")
    st.write("#")
    st.write("#")
    st.write("#")

if st.session_state.sb_query_history_range != "Choose a Date Range":
    df_query_history_range = pd.DataFrame(session.sql(f"""
                                                SELECT * 
                                                FROM (
                                                    SELECT
                                                        MAX(QUERY_ID) QUERY_ID
                                                        ,QUERY_TEXT
                                                        ,COUNT(QUERY_TEXT) NUM_OF_EXECUTIONS
                                                        ,MIN(START_TIME) FIRST_EXECUTION
                                                        ,MAX(END_TIME) LATEST_EXECUTION
                                                        ,ROUND(TIMEDIFF('minute', MIN(START_TIME), MAX(END_TIME)) / (NUM_OF_EXECUTIONS-1), 0) AVG_SCHEDULE_MINS
                                                        ,MAX(QUERY_TYPE) QUERY_TYPE
                                                        ,USER_NAME
                                                        ,MAX(WAREHOUSE_NAME) WAREHOUSE_NAME
                                                        ,MAX(WAREHOUSE_SIZE) WAREHOUSE_SIZE
                                                        ,MAX(WAREHOUSE_TYPE) WAREHOUSE_TYPE
                                                        ,MAX(CLUSTER_NUMBER) CLUSTER_NUMBER
                                                    FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
                                                    WHERE
                                                        (QUERY_TEXT ILIKE 'create or replace table%as select%'
                                                        OR QUERY_TEXT ILIKE 'insert overwrite into%')
                                                        AND LOWER(EXECUTION_STATUS) = 'success'
                                                        AND LOWER(USER_NAME) <> 'system'
                                                        AND END_TIME > DATEADD({date_time_part}, -{increment}, CURRENT_TIMESTAMP())
                                                    GROUP BY
                                                        QUERY_TEXT
                                                        ,USER_NAME
                                                ) qh
                                                WHERE qh.NUM_OF_EXECUTIONS > 1
                                                ;
                                                """).collect())

    
    #dynamically set data_editor height, based on number of rows in data frame
    df_query_history_range_height = int((len(df_query_history_range) + 1.5) * 35 + 3.5)

    #preview dataframe
    st.write("")
    st.subheader(f"Latest Non-Task commands ({st.session_state.sb_query_history_range})")
    st.dataframe(df_query_history_range, hide_index=True, height=df_query_history_range_height, use_container_width=True)

## STEP 5: Get command details

This step: compiles a list of commands eligible to be converted to dynamic tables, along with whether the target table is included in a data share.

In [ ]:
#check each command
list_eligible_cmds = []
list_ineligible_cmds = []

if not df_query_history_range.empty:
    for index, row in df_query_history_range.iterrows():
        db = ""
        sch = ""
        tbl = ""
    
        source_select = ""
    
        query_id = row["QUERY_ID"]
        query_text = sqlparse.format(row["QUERY_TEXT"], reindent=True, keyword_case="upper")
        schedule = 1 if row["AVG_SCHEDULE_MINS"] < 1 else row["AVG_SCHEDULE_MINS"]
        #query_type = row["QUERY_TYPE"]
        query_type = "CTAS" if row["QUERY_TYPE"].lower() == 'create_table_as_select' else "INSERT_OVERWRITE"
        warehouse = row["WAREHOUSE_NAME"]
    
        #use get_query_operator_stats to get target table name
        df_target_table_full = pd.DataFrame(session.sql(f"""SELECT
                                                            OPERATOR_ATTRIBUTES:table_name::varchar TARGET_TABLE
                                                            ,OPERATOR_ATTRIBUTES:table_names[0]::varchar TARGET_TABLES
                                                        FROM TABLE(GET_QUERY_OPERATOR_STATS('{query_id}')) 
                                                        WHERE LOWER(OPERATOR_TYPE) IN('insert', 'createtableasselect')
                                                        ;""").collect())
            
        target_table_full = df_target_table_full.iloc[0,0] if df_target_table_full.iloc[0,0] else df_target_table_full.iloc[0,1]
    
        if len(target_table_full.split(".")) == 4:
            acct = target_table_full.split(".")[0]
            db = target_table_full.split(".")[1]
            sch = target_table_full.split(".")[2]
            tbl = target_table_full.split(".")[3]
    
            target_table = f"{db}.{sch}.{tbl}"
            
        if len(target_table_full.split(".")) == 3:
            target_table = target_table_full
    
        #set share flag whether target is in a share:
        share_details = {}
        flag_target_shared = "Y" if target_table in list_objs else "N"
    
        share_details.update({"target_shared" : f"{flag_target_shared}"})
    
        if flag_target_shared == "Y":
            shares_target = []
    
            df_shared_objs_filtered = df_shared_objs.query(f"""object == '{target_table}'""")
    
            for index, row in df_shared_objs_filtered.iterrows():
                share_details.update({"object" : f"""{row["object"]}"""})
                share_details.update({"object_type" : f"""{row["object_type"]}"""})
                shares_target.append(row["share"])
                
            share_details.update({"shares" : shares_target})
    
        #create dynamic table DDL prefix        
        create_dt_ddl = f"""CREATE OR REPLACE DYNAMIC TABLE {target_table}_DT
                            TARGET_LAG = '{schedule} MINUTES'
                            WAREHOUSE = {warehouse}
                            COMMENT = '{{"origin":"sf_sit","name":"dt_conversion_non_task","version":{{"major":1, "minor":0}},"attributes":{{"source":"command", "type":"{query_type}"}}}}'
                            AS
                            """
        
        #check if query selects from a base object
        if re.search(r"(?s)(?=SELECT)(.*?\s+FROM.*)", query_text):
            #get source select statement
            source_select = re.search(r"(?s)(?=SELECT)(.*?\s+FROM.*)", query_text).group(1)
        
            #append source select statement to DT DDL 
            create_dt_ddl += source_select

            #check if create DT statement is valid using EXPLAIN
            try:
                explain_dt_statement = pd.DataFrame(session.sql(f"""EXPLAIN USING JSON {create_dt_ddl}""").collect()).iloc[0,0]
            except Exception as e:
                reason = str(e)

                #add command to ineligible list
                list_ineligible_cmds.append([query_type, query_text, reason])
            else:
                #add command to eligible list
                list_eligible_cmds.append([False, query_type, json.dumps([share_details], indent=2), query_text, create_dt_ddl])
        else:
            reason = "This command does not select from a base object."
            #add command to ineligible list
            list_ineligible_cmds.append([query_type, query_text, reason])

    
    st.write("")
    st.subheader("Ineligible Commands:")
    st.write("The following command(s) cannot be converted to Dynamic Tables")
    
    #create a dataframe from list_ineligible_cmds
    df_inelibible_cmd_clmns = ['Command Type'
                                 ,'Command DDL'
                                 ,'Reason'
                                ]
    
    df_inelibible_cmd = pd.DataFrame(list_ineligible_cmds, columns = df_inelibible_cmd_clmns)
    
    #dynamically set data_editor height, based on number of rows in data frame
    de_inelibible_cmd_height = int((len(df_inelibible_cmd) + 1.5) * 35 + 3.5)
    
    de_inelibible_cmd = st.dataframe(
        df_inelibible_cmd
        ,height=de_inelibible_cmd_height
        ,hide_index=True
        ,use_container_width=True
    )
    
    st.write("#")
    st.subheader("Eligible Commands:")
    st.write("Please choose scheduled command(s) to convert, using the `Convert` checkbox.  Any command selected will be converted in Step 6.")
    
    #create a dataframe from list_eligible_cmds
    df_convert_cmd_clmns = ['Convert'
                             ,'Command Type'
                             ,'Shared Objects'
                             ,'Command DDL'
                             ,'Dynamic Table DDL'
                            ]
    
    df_convert_cmd = pd.DataFrame(list_eligible_cmds, columns = df_convert_cmd_clmns)
    
    #dynamically set data_editor height, based on number of rows in data frame
    de_convert_cmd_height = int((len(df_convert_cmd) + 1.5) * 35 + 3.5)
    
    de_convert_cmd = st.data_editor(
        df_convert_cmd
        ,height=de_convert_cmd_height
        ,disabled=('Command Type','Shared Objects','Command DDL','Dynamic Table DDL')
        ,hide_index=True
        ,use_container_width=True
        ,num_rows="fixed"
    )

## STEP 6: Convert commands (optional)

This step converts the chosen commands from STEP 5 to Dynamic Tables

In [ ]:
df_selected_cmds = de_convert_cmd.query('Convert == True')

flag_disable_convert_btn = True

if True in set(de_convert_cmd['Convert']):
    flag_disable_convert_btn = False

btn_convert = st.button("Convert", disabled=flag_disable_convert_btn, type="primary")

if btn_convert:
    for index, row in df_selected_cmds.iterrows():
        ddl = row["Dynamic Table DDL"]

        for stmt in ddl.rstrip(';').split(";"):
            #get dt table name
            dt =  re.search(r"(?<=CREATE OR REPLACE DYNAMIC TABLE\s)(.*?)(?=\s+TARGET_LAG)", stmt).group(1)
            
            #create dynamic table(s)
            session.sql(f"""{stmt}""").collect()
            st.success(f"Dynamic Table: {dt} successfully created 🎉")

## STEP 7: Cleanup (optional)

This step can perform the following:
- removes target table(s) from any shares
- drops the target table(s)

In [ ]:
flag_disable_cleanup_btn = True
flag_disable_checkbox = True

if (True in set(de_convert_cmd['Convert'])):
    flag_disable_checkbox = False
    
st.write("")
st.checkbox("Remove target table(s) from shares", key="cb_remove_from_share", disabled=flag_disable_checkbox)
st.checkbox("Drop target table(s)", key="cb_drop_target_table", disabled=flag_disable_checkbox)

if (True in set(de_convert_cmd['Convert'])) and (st.session_state["cb_remove_from_share"] or st.session_state["cb_drop_target_table"]):
    flag_disable_cleanup_btn = False

btn_cleanup = st.button("Cleanup", disabled=flag_disable_cleanup_btn, type="primary")

if btn_cleanup:
    list_tbls_drop = []
    for index, row in df_selected_cmds.iterrows():
        shared_objs = json.loads(row["Shared Objects"])
        command_type = row["Command Type"]
        command_ddl = row["Command DDL"]

        if command_type == "CTAS":
            tbl = re.search(r"(?<=CREATE OR REPLACE TABLE\s)(.*?)(?=\s+AS)", command_ddl).group(1)
        if command_type == "INSERT_OVERWRITE":
            tbl = re.search(r"(?<=INSERT OR OVERWRITE INTO\s)(.*?)(?=\s+SELECT)", command_ddl).group(1)
        
        if st.session_state["cb_remove_from_share"]:
            #REMOVE FROM SHARE(S)
            for dict_obj in shared_objs:
                flag_shared = dict_obj["target_shared"]
    
                if flag_shared.lower() == 'y':
                    obj = dict_obj["object"]
                    obj_type = dict_obj["object_type"]
                    list_share = dict_obj["shares"]
    
                    #add obj/type to list
                    list_tbls_drop.append([obj, obj_type])
    
                    for share in list_share:
                        #remove obj from share
                        session.sql(f"""REVOKE SELECT ON {obj_type} {obj} FROM SHARE {share}""").collect()
                        st.success(f"{obj_type} {obj} successfully removed from share: {share}. 🎉")
                else:    
                    st.warning(f"Table: {tbl} is not currently shared", icon="⚠️")

        
        if st.session_state["cb_drop_target_table"]:
            session.sql(f"""DROP TABLE IF EXISTS {tbl}""").collect()
            st.success(f"Table: {tbl} successfully dropped. 🎉")

        st.divider()